# 4. PyLapi Advanced with Asana

`PyLapi` is built around resources.
A "resource" represents an entity stored on the backend server.

For example, Asana, a popular project management service, records things you need to do as "tasks", which can be in multiple "projects".
Both tasks and projects are examples of resources, with their data stored as backend entities and presented to the frontend through the API.

In this tutorial, we are going to use the [Asana API](https://developers.asana.com/reference/rest-api-reference) as an example to illustrate how the PyLapi resource model can effectively manage backend resources.

---
The Asana API is well designed, following best practices. It has consistent patterns and a simple and efficient data structure.

These are the patterns across all Asana resources:
1. Each resource entity is assigned a unique global ID called `gid`.
2. The resource name is the first segment of the API path.
    - Example: `projects` in `/projects/{gid}/project_statuses`.
3. Request data is sent in the request body's `data` attribute.
    - Example: `{"data": {"approval_status": "approved"}}` to update a task.
4. Response data is received in the `data` attribute.
    - Example: `{"data": {"gid": "1234567890123456", "assignee": {...}, ...}}` as a task resource data response.
5. The resource path alone as an API method returns an array of resources.
    - Example: `/projects` returns a list of projects.
6. A GET on the resource path followed by `{gid}` returns resource data for a single entity.
    - Example: `GET /projects/{gid}` gets a project.
7. A PUT on the resource path followed by `{gid}` updates an existing resource with attributes to update in the request body and returns the updated resource data.
    - Example: `PUT /projects/{gid}` updates a project and returns the updated project data.
8. A POST on the resource path creates a new resource entity with the required attributes in the request body and returns the resource data of the new entity.
    - Example: `POST /projects` creates a project and returns the new project data.
9. A DELETE on the resource path followed by `{gid}` deletes an existing resource.
    - Example: `DELETE /tasks/{gid}` deletes a task.
10. Query parameters are used to identify the resource and filter the outcome.
    - Example: `/tasks?project=1234567890123456&opt_fields=assignee.name,due_on` lists all tasks of a project, each with the assignee's name and the due date.

## Once and for all

Based on the 10 patterns of the Asana API design, we can create some common API methods in the root API class, simplifying the interface and reducing the learning curve for frontend programmers.

Here is the `aAPI` root class implementing the Asana patterns, commented with pattern reference number.

In [1]:
from pylapi import PyLapi

class aAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://app.asana.com/api/1.0"
        # 1. {gid} global ID
        self._resource_attrs.update({"gid": "$.gid"})

    # 4. "give" to return resource data
    # 5. GET (default) with no resource path lists resource entities
    @PyLapi.resource_method(give="$.data")
    def list(self): pass

    # 1. {gid} global ID
    # 6. GET (default) to get resource data
    # 4. "give" to return resource data
    # 4. "load" data into the resource object
    @PyLapi.resource_method(method_route="{gid}", give="$.data", load="$.data")
    def load(self): pass

    # 1. {gid} global ID
    # 7. PUT to update the resource entity
    # 3. "send" to format the request body
    # 4. "give" to return resource data
    # 4. "load" data into the resource object
    @PyLapi.resource_method(method_route="{gid}", http_method="PUT", send={"data": "$"}, give="$.data", load="$.data")
    def update(self): pass

    # 8. POST to create a new resource entity
    # 3. "send" to format the request body
    # 4. "give" to return resource data
    # 4. "load" data into the resource object
    @PyLapi.resource_method(http_method="POST", send={"data": "$"}, give="$.data", load="$.data")
    def create(self): pass

    # 1. {gid} global ID
    # 9. DELETE to delete a new resource entity
    @PyLapi.resource_method(method_route="{gid}", http_method="DELETE", give=None)
    def delete(self): pass

Now let us define two resources, project and task, for demonstration purposes.

In [2]:
@aAPI.resource_class(resource_name="project", resource_base_route="projects")
class ProjectResource(aAPI): pass

@aAPI.resource_class(resource_name="task", resource_base_route="tasks")
class TaskResource(aAPI): pass

---
Before using the Asana API, you need to [sign up with Asana](https://asana.com/create-account), create a [personal access token](https://app.asana.com/0/my-apps), and save it to `._asecret` under the current directory for authenticating the API.

IMPORTANT: Please store your personal access token securely without exposing it to any output or repositories.

In [3]:
# Authenticate with your Asana personal access token saved in `._osecret`.
aAPI.auth(open("._asecret", "r").readlines()[0].strip())

All Asana accounts come with an "Asana" project with six tasks. Let us find out what they are.

In [4]:
# Listing projects
project = aAPI.resource("project")
project_list = project.list()

if not project.response_ok():
    print(f"API response: {project_list}")
else:
    # Find the gid (global ID) of the "Asana" project
    project_gid = ([_["gid"] for _ in project_list if _["name"] == "Asana"])[0]

    # List all tasks of the project
    task = aAPI.resource("task")
    task_list = task.list(project=project_gid)

    # Print all tasks in name order
    task_list.sort(key=lambda _: _["name"])
    for _ in task_list: print(f'{_["gid"]}: {_["name"]}')

1204597119276197: 1️⃣ First: Get started using My Tasks
1204597119276199: 2️⃣ Second: Find the layout that's right for you
1204597119276201: 3️⃣ Third: Get organized with sections
1204597119276203: 4️⃣ Fourth: Stay on top of incoming work
1204597119276205: 5️⃣ Fifth: Save time by collaborating in Asana
1204597119276207: 6️⃣ Sixth: Make work manageable
1205166645261614: Test Task 2


---
You can also access the low-level API request and response if you so choose.

In [5]:
import json

print("Request:")
print(task.request_http_method)
print(json.dumps(task.request, indent=4))

print("")
print("Response:")
print(task.response)
print(json.dumps(task.response_data, indent=4))

Request:
GET
{
    "url": "https://app.asana.com/api/1.0/tasks",
    "headers": {
        "Authorization": "Bearer <api_auth>"
    },
    "params": {
        "project": "1204597201454351"
    }
}

Response:
<Response [200]>
[
    {
        "gid": "1204597119276197",
        "name": "1\ufe0f\u20e3 First: Get started using My Tasks",
        "resource_type": "task",
        "resource_subtype": "default_task"
    },
    {
        "gid": "1204597119276199",
        "name": "2\ufe0f\u20e3 Second: Find the layout that's right for you",
        "resource_type": "task",
        "resource_subtype": "default_task"
    },
    {
        "gid": "1204597119276201",
        "name": "3\ufe0f\u20e3 Third: Get organized with sections",
        "resource_type": "task",
        "resource_subtype": "default_task"
    },
    {
        "gid": "1204597119276203",
        "name": "4\ufe0f\u20e3 Fourth: Stay on top of incoming work",
        "resource_type": "task",
        "resource_subtype": "default_task"
  

## Loading Resource Data

So far we are only listing resources but not loading any resource data yet, so the resource objects are still empty.

In [6]:
print(project)
print(task)

{}
{}


To load project resource data, we call `project.load()`. (The full printout of the `project` object would be long. You may uncomment the last print statement if you want to see all project data attributes.)

In [7]:
project.load(project_gid)
print(project.gid) # .gid is mapped to .data.gid
print(project.data.name)
print(project.data.owner.name)
# print(project) # Uncomment to print the full project object

1204597201454351
Asana
Jacky Ko


In [8]:
task.load(task_list[0]["gid"])
print(task.gid) # .gid is mapped to .data.gid
print(task.data.name)
print(task.data.assignee_section.name)
print(task.data.tags)
# print(task) # Uncomment to print the full task object

1204597119276197
1️⃣ First: Get started using My Tasks
Recently assigned
[{'gid': '1205189493487562', 'name': 'Deliverable', 'resource_type': 'tag'}, {'gid': '1205189493487563', 'name': 'Social', 'resource_type': 'tag'}]


Similarly, you may load the first task on the list into the `task` resource object.

---
In the next tutorial, we are going to discuss [Resource Data](5.%20Resource%20Data.ipynb) in more detail.

## End of page